In [1]:
import os
import pprint
import tensorflow as tf
import math
import time

import random
import numpy as np
from past.builtins import xrange

from data import read_data
import sys
sys.path.append('../tfmodels')
from memory_model import *



In [2]:
data_dir = 'data'
data_name = 'ptb'
checkpoint_dir = 'checkpoints'

count = []
word2idx = {}

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

train_data = read_data('%s/%s.train.txt' % (data_dir, data_name), count, word2idx)
valid_data = read_data('%s/%s.valid.txt' % (data_dir, data_name), count, word2idx)
test_data = read_data('%s/%s.test.txt' % (data_dir, data_name), count, word2idx)

idx2word = dict(zip(word2idx.values(), word2idx.keys()))
nwords = len(word2idx)


Read 929589 words from data/ptb.train.txt
Read 73760 words from data/ptb.valid.txt
Read 82430 words from data/ptb.test.txt


In [3]:
maxlen = 100
train_samples = len(range(0, len(train_data) - maxlen, 1))
valid_samples = len(range(0, len(valid_data) - maxlen, 1))
samples = train_samples+valid_samples
idxs = np.arange(0, samples)
train_idxs = idxs[0:train_samples]
test_idxs = idxs[train_samples:]
previous_words = np.empty(shape=(samples, maxlen), dtype=np.int32)
next_words = np.empty(shape=(samples), dtype=np.int32)
global_step = 0

for i in range(0, len(train_data) - maxlen, 1):
    previous_words[global_step] = train_data[i: i + maxlen]
    next_words[global_step] = train_data[i + maxlen]
    global_step += 1

for i in range(0, len(valid_data) - maxlen, 1):
    previous_words[global_step] = valid_data[i: i + maxlen]
    next_words[global_step] = valid_data[i + maxlen]
    global_step += 1


In [4]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", nwords, "term number in input sequence(zero mask)")
flags.DEFINE_integer("mem_size",maxlen, "the memory length of input sequence")
flags.DEFINE_integer("nhop", 6, "the number of hop layers")
flags.DEFINE_integer("embedding_size", 150, "word embedding size")
flags.DEFINE_float("init_hid", 0.1, "init_hid for q")
flags.DEFINE_float("init_std", 0.05, "init_std for training variable")

flags.DEFINE_float("linear_ratio", 0.5, "keep probability of drop out [0.9]")
flags.DEFINE_float("learning_rate", 0.001, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 1024, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 50.0, "clip gradients to this norm")
flags.DEFINE_integer("n_epochs", 100, "number of epoch to use during training")
flags.DEFINE_boolean("epoch_save", False, "save checkpoint or not in each epoch")
flags.DEFINE_integer("print_step", 500, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", True, "use lr annealing or not after each epoch [False]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS


In [7]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    mem_model = MemN2NModel(FLAGS, session)
    mem_model.build_model()
    mem_model.build_model_summary()
    display(mem_model.model_summary())
    #mem_model.run([previous_words,next_words], train_idxs, test_idxs)

Initializing


variable_name variable_shape  parameters
0                   global/Variable:0             []         1.0
1  gpu/memory/Ain_c/embedding_table:0   [10000, 150]   1500000.0
2                gpu/memory/Ain_t/W:0     [100, 150]     15000.0
3  gpu/memory/Cin_c/embedding_table:0   [10000, 150]   1500000.0
4                gpu/memory/Cin_t/W:0     [100, 150]     15000.0
5          gpu/momory_hops/hops_h/W:0     [150, 150]     22500.0
6               gpu/prediction/fc/W:0   [150, 10000]   1500000.0